# Apriori

## Importing the libraries

In [6]:
!pip install apyori #para instalar modulo apyori (scikitlearn n tem tudo. nem o colab.)

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Data Preprocessing

In [8]:
dataset = pd.read_csv('Market_Basket_Optimisation.csv', header = None) #nota - como a primeira linha vai ter logo informação pertinente, metemos header none (pois não tem header)
transactions = [] #empty list
#for para meter todas as transacoes presentes no dataset
for i in range(0, 7501): #de 1 a 7500, pois o 7501 não irá fazer nada
  transactions.append([str(dataset.values[i,j]) for j in range(0, 20)]) #para fazer uma lista de listas. Para ficar com todos os produtos identificados
#0 a 20 porque ja sabemos que o maior numero de produtos por transacao é 19 (metemos logo na primeira linha)
#O que ira fazer é -> para cada row (transacao) percorre os produtos todos e guarda numa lista
#str -> para ficar com os produtos em string.

## Training the Apriori model on the dataset

In [9]:
from apyori import apriori
#train apriori model and get the final rules (with apropriate confidence)
rules = apriori(transactions = transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2, max_length = 2)
#dataset -> vao ser as transactions
#suporte minimo -> numero de transacoes A/B dividido pelo total. Queremos no minimo
  #3 transacoes com esses dois produtos que teremos associacao.
  # 3 * 7 (uma semana) / 7501 (numero de transacoes numa semana) = 0,003
#confianca minima -> variável caso a caso mas vamos meter 0,2.
#minimum lift -> mede a relevancia de uma rule ((Confidence a dividir pelo suporte)). Queremos, no minimo, 3. É common sense
#min length e max length -> Minimo length de elementos na rule. Max o maximo. Como queremos ver produto A e produto B (melhores pares), metemos 2

## Visualising the results

### Displaying the first results coming directly from the output of the apriori function

In [10]:
results = list(rules)
results

[RelationRecord(items=frozenset({'chicken', 'light cream'}), support=0.004532728969470737, ordered_statistics=[OrderedStatistic(items_base=frozenset({'light cream'}), items_add=frozenset({'chicken'}), confidence=0.29059829059829057, lift=4.84395061728395)]),
 RelationRecord(items=frozenset({'mushroom cream sauce', 'escalope'}), support=0.005732568990801226, ordered_statistics=[OrderedStatistic(items_base=frozenset({'mushroom cream sauce'}), items_add=frozenset({'escalope'}), confidence=0.3006993006993007, lift=3.790832696715049)]),
 RelationRecord(items=frozenset({'pasta', 'escalope'}), support=0.005865884548726837, ordered_statistics=[OrderedStatistic(items_base=frozenset({'pasta'}), items_add=frozenset({'escalope'}), confidence=0.3728813559322034, lift=4.700811850163794)]),
 RelationRecord(items=frozenset({'honey', 'fromage blanc'}), support=0.003332888948140248, ordered_statistics=[OrderedStatistic(items_base=frozenset({'fromage blanc'}), items_add=frozenset({'honey'}), confidence=0

### Putting the results well organised into a Pandas DataFrame

In [11]:
#Para arrumar a lista de rules e depois arranjar.
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results] #produto left hand side
    rhs         = [tuple(result[2][0][1])[0] for result in results] #produto right hand side
    supports    = [result[1] for result in results] #suporte
    confidences = [result[2][0][2] for result in results] #confianca
    lifts       = [result[2][0][3] for result in results] #lifts
    return list(zip(lhs, rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])

### Displaying the results non sorted

In [12]:
resultsinDataFrame #mostra a tabela que criamos no anterior.
#Como ler? Se compraram Left Hand Side, a probabilidade de comprarem Right Hand Side é -> Confidence.
#Lift -> relevancia da rule (Confidence a dividir pelo suporte)

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,light cream,chicken,0.004533,0.290598,4.843951
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
2,pasta,escalope,0.005866,0.372881,4.700812
3,fromage blanc,honey,0.003333,0.245098,5.164271
4,herb & pepper,ground beef,0.015998,0.323450,3.291994
5,tomato sauce,ground beef,0.005333,0.377358,3.840659
6,light cream,olive oil,0.003200,0.205128,3.114710
7,whole wheat pasta,olive oil,0.007999,0.271493,4.122410
8,pasta,shrimp,0.005066,0.322034,4.506672


### Displaying the results sorted by descending lifts

In [13]:
resultsinDataFrame.nlargest(n = 10, columns = 'Lift')
#top 10 por lift

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
3,fromage blanc,honey,0.003333,0.245098,5.164271
0,light cream,chicken,0.004533,0.290598,4.843951
2,pasta,escalope,0.005866,0.372881,4.700812
8,pasta,shrimp,0.005066,0.322034,4.506672
7,whole wheat pasta,olive oil,0.007999,0.271493,4.122410
5,tomato sauce,ground beef,0.005333,0.377358,3.840659
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
4,herb & pepper,ground beef,0.015998,0.323450,3.291994
6,light cream,olive oil,0.003200,0.205128,3.114710
